<h3>Token</h3>
<h4>Analisador Lexico - AFD </h4>

In [45]:
T_KEYWORD = "<keyword %s>"
T_OP = "<op %s>"
T_INT = "<int %s>"
T_STRING = "<string %s>"
T_IDENTIF = "<id %s>"
T_SPECIAL = "<special %s>"
T_PUNCT = "<punct %s>"
T_DOT = "<dot>"
T_CONDITIONAL_OP = "<conditional_op %s>"
T_COMMENT = "<comment>"

class Token():
    def __init__(self, tipo, valor=None):
        self.tipo = tipo
        self.valor = valor

    def __str__(self):
        return f"Token(tipo={self.tipo}, valor={self.valor})"

    def __repr__(self):
        return self.__str__()

class StopExecution(Exception):
    def _render_traceback_(self):
        pass

def tokenize_line(line, line_number):
    tokens = []
    i = 0
    n = len(line)

    while i < n:
        if line[i].isspace():
            i += 1
        elif line[i] == '#':  # Comment
            while i < n:
                i += 1
            break
            # start = i
            # while i < n:
            #     i += 1
            # tokens.append(Token("T_COMMENT", line[start:i]))
            break
        elif line[i].isalpha():
            start = i
            while i < n and (line[i].isalnum() or line[i] == '_'):
                i += 1
            word = line[start:i]
            if word in ["var", "func", "if", "elif", "else", "return", "object", "init"]:
                tokens.append(Token("T_KEYWORD", word))
            elif word in ["true", "false", "null", "end", "main"]:
                tokens.append(Token("T_SPECIAL", word))
            else:
                tokens.append(Token("T_IDENTIF", word))
        elif line[i].isdigit():
            start = i
            while i < n and line[i].isdigit():
                i += 1
            tokens.append(Token("T_INT", line[start:i]))
        elif line[i] == '"':
            start = i
            i += 1
            while i < n and line[i] != '"':
                i += 1
            if i >= n:
                print(f"Erro: String não fechada na linha {line_number}")
                raise StopExecution
            i += 1
            tokens.append(Token("T_STRING", line[start:i]))
        # elif line[i] in "=<>!+-*/":
        #     start = i
        #     i += 1
        #     if i < n and line[i] == "=":
        #         i += 1
        #     tokens.append(Token("T_OP", line[start:i]))
        elif line[i] in "=<>!":
            start = i
            i += 1
            if i < n and line[i] == "=":
                i += 1
                tokens.append(Token("T_OP", line[start:i]))
            else:
                tokens.append(Token("T_OP", line[start:i]))
        elif line[i] in "+-*/":
            tokens.append(Token("T_OP", line[i]))
            i += 1
        elif line[i] in "(),[]{}":
            tokens.append(Token("T_PUNCT", line[i]))
            i += 1
        elif line[i] in "?:":
            tokens.append(Token("T_CONDITIONAL_OP", line[i]))
            i += 1
        elif line[i] == '.':
            tokens.append(Token("T_DOT"))
            i += 1
        else:
            print(f"Erro: Caractere não reconhecido '{line[i]}' na linha {line_number}")
            raise StopExecution
    return tokens

def tokenize():

    try:
        token_total = [];
        with open('codigo2.x', 'r') as f:
            lines = f.read().splitlines()

        line_number = 0
        for line in lines:
            line_number += 1
            tokens = tokenize_line(line, line_number)
            token_total.extend(tokens)
            #print(tokens)

        return token_total

    except StopExecution:
        print("Execução parada devido a erro.")

<h3>Analisador Sinatico - Parser </h3>

In [46]:
class Parser:
    def __init__(self, tokens):
        self.tokens = tokens
        self.current_idx = 0  
        self.current_token = self.tokens[0] if tokens else None
        self.symbol_table = {}

    def get_next_token(self):
        self.current_idx += 1
        if self.current_idx < len(self.tokens):
            self.current_token = self.tokens[self.current_idx]
        else:
            self.current_token = None

    def eat(self, token_type):
        print(f"Consumindo token: {self.current_token.valor} do tipo {self.current_token.tipo}")
        if self.current_token.tipo == token_type:
            self.get_next_token()
        else:
            raise Exception(f"Erro de sintaxe. Esperado: {token_type}. Recebido: {self.current_token.tipo} - Valor: {self.current_token.valor}")

    def factor(self):
        """factor : INT | IDENTIF | ( expr )"""
        token = self.current_token
        if token.tipo == 'T_INT':
            self.eat('T_INT')
            return int(token.valor)
        elif token.tipo == 'T_IDENTIF':
            self.eat('T_IDENTIF')
            return self.symbol_table.get(token.valor, None) 
        elif token.tipo == 'T_PUNCT' and token.valor == '(':
            self.eat('T_PUNCT')
            result = self.expr()
            self.eat('T_PUNCT')
            return result

    def term(self):
        """term : factor ((MUL | DIV) factor)*"""
        result = self.factor()

        while self.current_token is not None and self.current_token.tipo in ('T_OP') and self.current_token.valor in ['*', '/']:
            token = self.current_token
            if token.valor == '*':
                self.eat('T_OP')
                result *= self.factor()
            elif token.valor == '/':
                self.eat('T_OP')
                result /= self.factor()

        return result

    def expr(self):
        """expr : term ((PLUS | MINUS) term)*"""
        result = self.term()

        while self.current_token is not None and self.current_token.tipo in ('T_OP') and self.current_token.valor in ['+', '-']:
            token = self.current_token
            if token.valor == '+':
                self.eat('T_OP')
                result += self.term()
            elif token.valor == '-':
                self.eat('T_OP')
                result -= self.term()

        return result

    def assignment(self):
        """IDENTIF EQUALS expr"""
        var_name = self.current_token.valor
        self.eat('T_IDENTIF')
        self.eat('T_OP')
        var_value = self.expr()
        self.symbol_table[var_name] = var_value
    
    def conditional_statement(self):
        """ if/elif/else statement """
        self.eat('T_KEYWORD')  # Consumir o "if" ou "elif"
        condition = self.expr()  # Avaliar a expressão condicional
        self.eat('T_PUNCT')  # Consumir o '{'
        
        if condition:
            self.start()  # Processar o bloco de código dentro do if/elif
        else:
            # pular o bloco de código e possivelmente ir para um "elif" ou "else"
            while self.current_token.tipo != 'T_PUNCT' or self.current_token.valor != '}':
                self.get_next_token()

        self.eat('T_PUNCT')  # Consumir o '}'

        if self.current_token and self.current_token.valor == "elif":
            self.conditional_statement()
        elif self.current_token and self.current_token.valor == "else":
            self.eat('T_KEYWORD')  # Consumir o "else"
            self.eat('T_PUNCT')  # Consumir o '{'
            self.start()  # Processar o bloco de código dentro do else
            self.eat('T_PUNCT')  # Consumir o '}'

    def loop_statement(self):
        """ while statement"""
        self.eat('T_KEYWORD')  # Consumir o "while"
        condition = self.expr()  # Avaliar a expressão condicional inicialmente

        while condition:
            self.eat('T_PUNCT')  # Consumir o '{'
            self.start()  # Processar o bloco de código dentro do loop
            self.eat('T_PUNCT')  # Consumir o '}'
            
            # Reavaliar a condição depois de processar o bloco de código
            condition = self.expr()

    def function_definition(self):
        """ func IDENTIFIER (...) """
        self.eat('T_KEYWORD')  # Consumir o "func"
        func_name = self.current_token.valor
        self.eat('T_IDENTIF')
        self.eat('T_PUNCT')  # Consumir o '('

        # Lista de parâmetros (opcional)
        parameters = []
        while self.current_token.tipo != 'T_PUNCT' or self.current_token.valor != ')':
            parameters.append(self.current_token.valor)
            self.eat('T_IDENTIF')

            # Se houver uma vírgula, é porque há mais parâmetros
            if self.current_token.valor == ',':
                self.eat('T_PUNCT')  # Consumir a ','

        self.eat('T_PUNCT')  # Consumir o ')'

        # Aqui, você pode adicionar a função à sua tabela de símbolos ou processá-la de alguma outra forma.
        # Atualmente, apenas consumimos o corpo da função sem processá-lo
        self.eat('T_PUNCT')  # Consumir o '{'
        while self.current_token.tipo != 'T_PUNCT' or self.current_token.valor != '}':
            self.get_next_token()
        self.eat('T_PUNCT')  # Consumir o '}'

    def print_command(self):
        """ print( ... ) """
        self.eat('T_KEYWORD')  # Consumir o "print"
        self.eat('T_PUNCT')  # Consumir o '('

        while self.current_token.tipo != 'T_PUNCT' or self.current_token.valor != ')':
            if self.current_token.tipo == 'T_STRING':
                print(self.current_token.valor)
                self.eat('T_STRING')
            else:
                print(self.expr())

            if self.current_token and self.current_token.valor == ',':
                self.eat('T_PUNCT')  # Consumir a ','

        self.eat('T_PUNCT')  # Consumir o ')'

    def special_function_definition(self):
        """ main() ou init() """
        # Adicionar lógica para tratar funções especiais, se necessário.
        pass

    def object_definition(self):
        """ object IDENTIFIER """
        # Adicionar lógica para tratar a definição de objetos.
        pass

    def statement(self):
        if self.current_token.valor in ["if", "elif", "else"]:
            self.conditional_statement()
        elif self.current_token.valor == "while":
            self.loop_statement()
        elif self.current_token.valor == "func":
            self.function_definition()
        elif self.current_token.valor in ["main", "init"]:
            self.special_function_definition()
        elif self.current_token.valor == "object":
            self.object_definition()
        elif self.current_token.valor == "print":
            self.print_command()
        else:
            self.assignment()

    def start(self):
        while self.current_token:
            self.statement()


<h3>Compilador</h3>

In [49]:
def main():
    filename = 'codigo2.x'
    print(f"Lendo o arquivo {filename} ...")

    arquivo = open(filename)
    for l in arquivo.readlines():
        l = l.replace('\n','') # remove a quebra de linha
        print(l)

    # Tokenização
    print("\nTokenização:")
    tokens = tokenize()
    print("Tokens:")
    for i, token in enumerate(tokens):
        print(f"{i+1:03}. {token}")
    
    # Análise Sintática (Parser)
    print("\nAnálise Sintática:")
    parser = Parser(tokens)
    try:
        parser.start()
        print("Análise sintática concluída com sucesso!")
    except Exception as e:
        print(f"Erro no parser: {e}")

    states = {
        'tokens': tokens,
        'symbol_table': parser.symbol_table
    }
    print("\nEstados salvos:", states)

if __name__ == '__main__':
    main()

Lendo o arquivo codigo2.x ...
# ANT
func home(){
    var a = 1
    var b = 2
}

Tokenização:
Tokens:
001. Token(tipo=T_KEYWORD, valor=func)
002. Token(tipo=T_IDENTIF, valor=home)
003. Token(tipo=T_PUNCT, valor=()
004. Token(tipo=T_PUNCT, valor=))
005. Token(tipo=T_PUNCT, valor={)
006. Token(tipo=T_KEYWORD, valor=var)
007. Token(tipo=T_IDENTIF, valor=a)
008. Token(tipo=T_OP, valor==)
009. Token(tipo=T_INT, valor=1)
010. Token(tipo=T_KEYWORD, valor=var)
011. Token(tipo=T_IDENTIF, valor=b)
012. Token(tipo=T_OP, valor==)
013. Token(tipo=T_INT, valor=2)
014. Token(tipo=T_PUNCT, valor=})

Análise Sintática:
Consumindo token: func do tipo T_KEYWORD
Consumindo token: home do tipo T_IDENTIF
Consumindo token: ( do tipo T_PUNCT
Consumindo token: ) do tipo T_PUNCT
Consumindo token: { do tipo T_PUNCT
Consumindo token: } do tipo T_PUNCT
Análise sintática concluída com sucesso!

Estados salvos: {'tokens': [Token(tipo=T_KEYWORD, valor=func), Token(tipo=T_IDENTIF, valor=home), Token(tipo=T_PUNCT, valor